In [17]:
import pandas as pd

distance = 25
params = 0.99

dataPath = '../../data/'
filePath = f'{dataPath}/processed/adjacency_matrix.csv'
distance_matrix = pd.read_csv(filePath, index_col=0)


distance_threshold = distance
local_params = params
adj_params = 1 - local_params

dm_with_threshhold = distance_matrix[distance_matrix < distance_threshold]


# dm_with_threshhold의 모든 가중치를 역수로 취하기.
# 가중치가 0일경우 1로 바꾸기.

dm_with_threshhold = dm_with_threshhold.replace(0, 1)
dm_with_threshhold = 1 / dm_with_threshhold


import numpy as np

# 각 행의 합을 계산
row_sums = dm_with_threshhold.sum(axis=1) - 1


In [18]:
# 각 인접 도시와의 가중치를 총합으로 나눔
normalized_df = dm_with_threshhold.div(row_sums, axis=0) * adj_params

In [19]:
# 대각선 행은 0.7으로 만들기

# 대각선 행 중 inf가 아닌 행은 0.7으로 만들기

for i in range(len(normalized_df)):
    if normalized_df.iloc[i, i] != np.inf:
        normalized_df.iloc[i, i] = local_params
    else:
        normalized_df.iloc[i, i] = 1



In [20]:
normalized_df.to_csv(f'{dataPath}/processed/weighted_df/normalized_df_{distance}_{params}.csv')

In [21]:
df = pd.read_csv(f'{dataPath}/processed/no_NaN_dataset_final.csv')

In [22]:
def find_cityname_in_df(df, cityname, year):
    # cityname에서 첫 두글자는 시도에, 나머지는 시군구에 저장함
    
    sido = cityname[:2]
    sigungu = cityname[2:]
    
    # sido와 sigungu를 모두 포함하는 행 찾기 꼭 일치하지 않아도 되고 일부만 포함해도 됨
    
    sido_df = df[df['행정구역(시도)'].str.contains(sido)]
    # sigungu_df = sido_df[sido_df['행정구역(시군구)'].str.contains(sigungu)]
    sigungu_df = sido_df[sido_df['행정구역(시군구)'] == sigungu]
    
    
    result = sigungu_df[sigungu_df['연도'] == year]
    
    if len(result) == 0:
        sido = cityname[:2]
        sigungu = cityname[2:]
        
        sido_df = df[df['행정구역(시도)'].str.contains(sido)]
        
        sigungu_df = sido_df[sido_df['행정구역(시군구)'].str.contains(sigungu)]
        
        result = sigungu_df[sigungu_df['연도'] == year]
        
    if len(result) != 1:
        print("길이는 : " + str(len(result)))
        print(result.head())

    return result
        
        


In [23]:
def find_normalizedf_row(do, city):
    return normalized_df.loc[do + city]


from tqdm import tqdm

processed_df = df.copy()
numeric_columns = df.select_dtypes(include=[np.number]).columns
numeric_columns = numeric_columns.drop('학령인구(명)')
error_city_list = []


for i in tqdm(range(len(df))):
    try:
        parts = df.iloc[i]['행정구역(시도)'].split()
        do = parts[0][:2]
        city = df.iloc[i]['행정구역(시군구)']
        year = df.iloc[i]['연도']

        adj_matrix = find_normalizedf_row(do, city)
        row_list = []
        
            
        for j in range(len(adj_matrix)):
            try:

                adj_city_name = adj_matrix.index[j]
                adj_city_weight = adj_matrix.iloc[j]
                if pd.isna(adj_city_weight):
                    continue
                
                

                row = find_cityname_in_df(df, adj_city_name, year)
                
                #tmp_df.insert(row)
                row[numeric_columns] *= adj_city_weight

                
                if(row.empty):
                    error_city_list.append(adj_city_name)
                    continue
                
                row_list.append(row)
            except KeyError:
                print("error 2: " + adj_matrix.index[j])
                continue
        
                
        # Sum all the rows in row_list
        new_row = pd.concat(row_list)

        
        new_row = new_row[numeric_columns]
        new_row_sum = new_row.sum()

        

        # Use .loc to avoid SettingWithCopyWarning
        processed_df.loc[processed_df.index[i], numeric_columns] = new_row_sum

    except Exception as e:
        print(f"error 3:  - {str(e)}")
        print(do + " " + city)
        continue


processed_df['연도'] = processed_df['연도'].round().astype(int)

  0%|          | 0/2899 [00:00<?, ?it/s]/var/folders/39/yhddl2n53db5kx8yl47wsn_m0000gn/T/ipykernel_48544/2421551742.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2010.0000000000002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  processed_df.loc[processed_df.index[i], numeric_columns] = new_row_sum
/var/folders/39/yhddl2n53db5kx8yl47wsn_m0000gn/T/ipykernel_48544/2421551742.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '172726.33930667822' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  processed_df.loc[processed_df.index[i], numeric_columns] = new_row_sum
/var/folders/39/yhddl2n53db5kx8yl47wsn_m0000gn/T/ipykernel_48544/2421551742.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Valu

In [ ]:
processed_df.to_csv(f'{dataPath}/processed/distance_df/{distance}_{params}.csv', index=False)

In [ ]:
processed_df.head()

,행정구역(시도),행정구역(시군구),연도,총인구수(명),유치원 수,초등학교 수,출생건수,사망건수,혼인건수,이혼건수,학령인구(명),총인구수-학령인구(명),행정구역
0,서울특별시,종로구,2010,170795.692385,17.022537,14.010845,1109.477804,852.694270,1012.623666,396.456493,29164,141588.857080,서울특별시 종로구
1,서울특별시,중구,2010,133082.052802,13.026904,12.013005,1172.410002,671.889758,1050.589448,378.475413,19628,113400.649423,서울특별시 중구
2,서울특별시,용산구,2010,245006.912106,19.021620,15.010422,2313.389756,1086.465502,1784.941162,528.338542,37318,207651.269059,서울특별시 용산구
3,서울특별시,성동구,2010,309184.241184,31.009008,20.004861,2886.721647,1226.377399,2306.361662,672.211435,51562,257599.756823,서울특별시 성동구
4,서울특별시,광진구,2010,375523.110954,41.999156,22.004677,3366.685849,1290.351565,2672.281749,811.108292,67422,308085.717129,서울특별시 광진구
